<a href="https://colab.research.google.com/github/ichko/ml-playground/blob/master/simple_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:
def random_polar(n, min_len, max_len):
    rad = np.random.uniform(0, 2 * np.pi, size=n)
    lengths = np.random.uniform(min_len, max_len, size=n)
    return np.array([np.sin(rad) * lengths, np.cos(rad) * lengths]).T

In [0]:
def plot_predictor(X, y, predict):
    h = .02  # step size in the mesh

    x_min, x_max = X[:, 0].min() - h, X[:, 0].max() + h
    y_min, y_max = X[:, 1].min() - h, X[:, 1].max() + h
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    fig, ax = plt.subplots()
    Z = predict(np.c_[xx.ravel(), yy.ravel()])

    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, 1 - Z)
    ax.axis('off')

    plt.scatter(X[:, 0], X[:, 1], c=y[:, 0])

In [0]:
## Data
num_per_class = 100

A = random_polar(num_per_class, 0, 0.5)
B = random_polar(num_per_class, 0.5, 1)

class_A = np.concatenate((A, np.ones((num_per_class, 1))), axis=1)
class_B = np.concatenate((B, np.zeros((num_per_class, 1))), axis=1)

data = np.concatenate((class_A, class_B), axis=0)
np.random.shuffle(data)

X, y = data[:, :2], data[:, 2:]
X.shape, y.shape

In [0]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_grad(z):
    s = sigmoid(z)
    return s * (1 - s)

In [0]:
num_inputs = X.shape[1]
num_outputs = y.shape[1]

num_hidden = 15
W1 = np.random.normal(size=(num_inputs, num_hidden)) * 3
b1 = np.zeros((1, num_hidden))

W2 = np.random.normal(size=(num_hidden, num_outputs)) * 3
b2 = np.zeros((num_outputs, 1))

In [0]:
## Tran
lr = 0.06
for step in range(10000):
    # forward
    z1 = X @ W1 + b1
    a1 = sigmoid(z1)
    
    z2 = a1 @ W2 + b2
    a2 = sigmoid(z2)
    
    J = np.sum((y - a2) ** 2) / 2
    
    # backward
    dJda2 = (a2 - y) / 2
    dJdz2 = dJda2 * sigmoid_grad(z2)
    dJdW2 = a1.T @ dJdz2
    dJdb2 = np.sum(dJdz2, axis=0, keepdims=True)
    
    dJda1 = dJdz2 @ W2.T
    dJdz1 = dJda1 * sigmoid_grad(z2)
    dJdW1 = X.T @ dJdz1
    dJdb1 = np.sum(dJdz1, axis=0, keepdims=True)

    # update
    W1 = W1 - lr * dJdW1
    b1 = b1 - lr * dJdb1
    W2 = W2 - lr * dJdW2
    b2 = b2 - lr * dJdb2

    # stats
    if step % 1000 == 0:
        print(J)

In [0]:
def predict(X):
    return sigmoid(sigmoid(X @ W1 + b1) @ W2 + b2)

plot_predictor(X, y, predict)